In [33]:
# Sample from https://www.kaggle.com/fkdplc/celeba-dcgan-for-generating-faces 
from keras import Input
from keras.layers import ReLU
from keras.layers import Dense, Reshape, LeakyReLU, Conv2D, Conv2DTranspose, Flatten, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
import os

#PIC_DIR = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/'
PIC_DIR = '../DATASETS/kaggle_celeba/img_align_celeba/img_align_celeba/'
len(os.listdir(PIC_DIR))

202599

In [2]:
IMAGES_COUNT = 10000

ORIG_WIDTH = 178
ORIG_HEIGHT = 208
diff = (ORIG_HEIGHT - ORIG_WIDTH) // 2

WIDTH = 128
HEIGHT = 128

crop_rect = (0, diff, ORIG_WIDTH, ORIG_HEIGHT - diff)

images = []
for pic_file in tqdm(os.listdir(PIC_DIR)[:IMAGES_COUNT]):
    pic = Image.open(PIC_DIR + pic_file).crop(crop_rect)
    pic.thumbnail((WIDTH, HEIGHT), Image.ANTIALIAS)
    images.append(np.uint8(pic))

images = np.array(images) / 255
images.shape

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 865.72it/s]


(10000, 128, 128, 3)

In [3]:
plt.figure(1, figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(images[i])
    plt.axis('off')
plt.show()

<Figure size 1000x1000 with 25 Axes>

In [76]:
LATENT_DIM = 64
CHANNELS = 3
INTERNAL_CONV_FEATURES = 128

def create_generator_old():
    gen_input = Input(shape=(LATENT_DIM, ))
    x = Dense(128 * 16 * 16)(gen_input)
    x = LeakyReLU()(x)
    x = Reshape((16, 16, 128))(x)
    x = Conv2D(256, 5, padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2DTranspose(INTERNAL_CONV_FEATURES, 4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2DTranspose(INTERNAL_CONV_FEATURES, 4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2DTranspose(INTERNAL_CONV_FEATURES, 4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Conv2D(INTERNAL_CONV_FEATURES * 2, 5, padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2D(INTERNAL_CONV_FEATURES * 2, 5, padding='same')(x)
    x = LeakyReLU()(x)
    x = Conv2D(CHANNELS, 7, activation='tanh', padding='same')(x)
    generator = Model(gen_input, x)
    return generator

def create_generator():
    gen_input = Input(shape=(LATENT_DIM, ))
    x = Dense(128 * 16 * 16)(gen_input)
    #x = BatchNormalization()(x)
    #x = ReLU()(x)
    # Hace falta? ver de sacar
    x = Reshape((16, 16, 128))(x)
    #x = BatchNormalization()(x)
    
    #x = Conv2D(256, 5, padding='same')(x)
    #x = ReLU()(x)
    
    x = Conv2DTranspose(INTERNAL_CONV_FEATURES, 4, strides=2, padding='same')(x)
    #x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2DTranspose(INTERNAL_CONV_FEATURES, 4, strides=2, padding='same')(x)
    #x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2DTranspose(INTERNAL_CONV_FEATURES, 4, strides=2, padding='same')(x)
    #x = BatchNormalization()(x)
    x = ReLU()(x)
    #x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
    #x = BatchNormalization()(x)
    #x = ReLU()(x)
    
    x = Conv2D(CHANNELS, 7, activation='tanh', padding='same')(x)
    generator = Model(gen_input, x)
    return generator

def create_discriminator():
    disc_input = Input(shape=(HEIGHT, WIDTH, CHANNELS))
    x = Conv2D(256, 3)(disc_input)
    #x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(INTERNAL_CONV_FEATURES, 4, strides=2)(x)
    #x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(INTERNAL_CONV_FEATURES, 4, strides=2)(x)
    #x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(INTERNAL_CONV_FEATURES, 4, strides=2)(x)
    #x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(INTERNAL_CONV_FEATURES, 4, strides=2)(x)
    #x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    x = Dense(1, activation='sigmoid')(x)
    
    discriminator = Model(disc_input, x)
    optimizer = RMSprop(clipvalue=1.0, decay=1e-8)
    discriminator.compile(optimizer=optimizer, loss='binary_crossentropy')
    return discriminator

In [77]:
generator = create_generator()
discriminator = create_discriminator()
discriminator.trainable = False

gan_input = Input(shape=(LATENT_DIM, ))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.summary()

optimizer = RMSprop(lr=.0001, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')

Model: "model_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_68 (InputLayer)        [(None, 64)]              0         
_________________________________________________________________
model_57 (Model)             (None, 128, 128, 3)       2935555   
_________________________________________________________________
model_58 (Model)             (None, 1)                 1323009   
Total params: 4,258,564
Trainable params: 2,935,555
Non-trainable params: 1,323,009
_________________________________________________________________


In [ ]:
import time

iters = 200000
batch_size = 16

RES_DIR = 'res6'
FILE_PATH = '%s/generated_%d.png'
if not os.path.isdir(RES_DIR):
    os.mkdir(RES_DIR)

CONTROL_SIZE_SQRT = 6
control_vectors = np.random.normal(size=(CONTROL_SIZE_SQRT**2, LATENT_DIM)) / 2

start = 0
d_losses = []
a_losses = []
images_saved = 0
for step in range(iters):
    start_time = time.time()
    latent_vectors = np.random.normal(size=(batch_size, LATENT_DIM))
    generated = generator.predict(latent_vectors)
    
    real = images[start:start + batch_size]
    combined_images = np.concatenate([generated, real])
    
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += .05 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images, labels)
    d_losses.append(d_loss)
    
    latent_vectors = np.random.normal(size=(batch_size, LATENT_DIM))
    misleading_targets = np.zeros((batch_size, 1))
    
    a_loss = gan.train_on_batch(latent_vectors, misleading_targets)
    a_losses.append(a_loss)
    
    start += batch_size
    if start > images.shape[0] - batch_size:
        start = 0
    
    if step % 50 == 49:
        gan.save_weights('gan.h5')
        
        print('%d/%d: d_loss: %.4f,  a_loss: %.4f.  (%.1f sec)' % (step + 1, iters, d_loss, a_loss, time.time() - start_time))
        
        control_image = np.zeros((WIDTH * CONTROL_SIZE_SQRT, HEIGHT * CONTROL_SIZE_SQRT, CHANNELS))
        control_generated = generator.predict(control_vectors)
        for i in range(CONTROL_SIZE_SQRT ** 2):
            x_off = i % CONTROL_SIZE_SQRT
            y_off = i // CONTROL_SIZE_SQRT
            control_image[x_off * WIDTH:(x_off + 1) * WIDTH, y_off * HEIGHT:(y_off + 1) * HEIGHT, :] = control_generated[i, :, :, :]
        im = Image.fromarray(np.uint8(control_image * 255))
        im.save(FILE_PATH % (RES_DIR, images_saved))
        images_saved += 1

50/200000: d_loss: 1.0279,  a_loss: 1.0683.  (1.0 sec)
100/200000: d_loss: 0.7685,  a_loss: 7.5075.  (1.0 sec)
150/200000: d_loss: 0.7612,  a_loss: 3.4071.  (1.0 sec)
200/200000: d_loss: 0.4938,  a_loss: 1.3327.  (0.6 sec)
250/200000: d_loss: 0.4134,  a_loss: 2.5525.  (1.0 sec)
300/200000: d_loss: 0.2916,  a_loss: 2.8903.  (1.0 sec)
350/200000: d_loss: 0.8064,  a_loss: 1.9272.  (1.0 sec)
400/200000: d_loss: 0.3815,  a_loss: 1.1934.  (1.0 sec)
450/200000: d_loss: 1.0116,  a_loss: 5.9681.  (1.0 sec)
500/200000: d_loss: 0.6323,  a_loss: 5.5803.  (1.0 sec)
550/200000: d_loss: 0.3685,  a_loss: 1.5417.  (1.0 sec)
600/200000: d_loss: 0.1761,  a_loss: 5.2450.  (1.0 sec)
650/200000: d_loss: 0.5252,  a_loss: 3.9535.  (1.0 sec)
700/200000: d_loss: 0.1764,  a_loss: 3.1216.  (1.0 sec)
750/200000: d_loss: 0.2639,  a_loss: 8.1411.  (1.0 sec)
800/200000: d_loss: 0.6971,  a_loss: 12.7454.  (1.0 sec)
850/200000: d_loss: 0.3221,  a_loss: 1.6481.  (0.6 sec)
900/200000: d_loss: 0.9628,  a_loss: 7.7849.  (0

7300/200000: d_loss: 1.5449,  a_loss: 0.7970.  (0.9 sec)
7350/200000: d_loss: 0.5740,  a_loss: 4.6865.  (1.0 sec)
7400/200000: d_loss: 0.2400,  a_loss: 4.6576.  (1.0 sec)
7450/200000: d_loss: 0.1996,  a_loss: 2.3467.  (1.0 sec)
7500/200000: d_loss: 0.1712,  a_loss: 4.6067.  (1.0 sec)
7550/200000: d_loss: 0.2969,  a_loss: 7.2784.  (1.0 sec)
7600/200000: d_loss: 0.3607,  a_loss: 3.8261.  (0.9 sec)
7650/200000: d_loss: 0.1838,  a_loss: 2.5287.  (1.0 sec)
7700/200000: d_loss: 0.1921,  a_loss: 3.3964.  (1.0 sec)
7750/200000: d_loss: 0.3885,  a_loss: 4.9433.  (1.3 sec)
7800/200000: d_loss: 0.1224,  a_loss: 4.3208.  (1.0 sec)
7850/200000: d_loss: 0.4477,  a_loss: 4.4532.  (1.0 sec)
7900/200000: d_loss: 0.5383,  a_loss: 2.0923.  (1.0 sec)
7950/200000: d_loss: 0.2378,  a_loss: 2.4098.  (0.9 sec)
8000/200000: d_loss: 0.3551,  a_loss: 2.6006.  (0.9 sec)
8050/200000: d_loss: 0.8439,  a_loss: 1.3938.  (1.0 sec)
8100/200000: d_loss: 0.3444,  a_loss: 3.6155.  (1.0 sec)
8150/200000: d_loss: 0.2290,  a

14450/200000: d_loss: 0.1933,  a_loss: 6.8472.  (1.0 sec)
14500/200000: d_loss: 0.0904,  a_loss: 5.3804.  (1.0 sec)
14550/200000: d_loss: 0.8133,  a_loss: 6.8999.  (1.2 sec)
14600/200000: d_loss: 0.1019,  a_loss: 4.8696.  (0.9 sec)
14650/200000: d_loss: 0.0899,  a_loss: 4.2545.  (0.9 sec)
14700/200000: d_loss: 0.1560,  a_loss: 4.6581.  (1.0 sec)
14750/200000: d_loss: 0.1074,  a_loss: 4.1217.  (1.0 sec)
14800/200000: d_loss: 0.1560,  a_loss: 2.6217.  (1.0 sec)
14850/200000: d_loss: 0.3848,  a_loss: 7.8725.  (1.0 sec)
14900/200000: d_loss: 0.0846,  a_loss: 5.3478.  (1.0 sec)
14950/200000: d_loss: 0.4444,  a_loss: 1.6476.  (0.6 sec)
15000/200000: d_loss: 0.1144,  a_loss: 5.0661.  (1.0 sec)
15050/200000: d_loss: 0.1522,  a_loss: 4.8700.  (1.0 sec)
15100/200000: d_loss: 0.1915,  a_loss: 5.2132.  (1.0 sec)
15150/200000: d_loss: 0.2437,  a_loss: 6.6705.  (1.0 sec)
15200/200000: d_loss: 0.6140,  a_loss: 7.4911.  (1.0 sec)
15250/200000: d_loss: 0.5460,  a_loss: 1.2897.  (1.0 sec)
15300/200000: 

21550/200000: d_loss: 1.1243,  a_loss: 6.7719.  (0.9 sec)
21600/200000: d_loss: 0.5538,  a_loss: 3.1312.  (1.0 sec)
21650/200000: d_loss: 0.5531,  a_loss: 5.6174.  (1.6 sec)
21700/200000: d_loss: 0.7500,  a_loss: 7.3590.  (1.0 sec)
21750/200000: d_loss: 0.1642,  a_loss: 3.3744.  (0.9 sec)
21800/200000: d_loss: 0.2220,  a_loss: 3.6118.  (1.0 sec)
21850/200000: d_loss: 0.2242,  a_loss: 4.3567.  (1.0 sec)
21900/200000: d_loss: 0.1435,  a_loss: 4.3785.  (1.0 sec)
21950/200000: d_loss: 0.2881,  a_loss: 6.3403.  (1.0 sec)
22000/200000: d_loss: 0.2297,  a_loss: 3.7287.  (1.2 sec)
22050/200000: d_loss: 0.0967,  a_loss: 3.7574.  (1.0 sec)
22100/200000: d_loss: 0.2452,  a_loss: 1.0664.  (1.0 sec)
22150/200000: d_loss: 0.2429,  a_loss: 4.4354.  (1.0 sec)
22200/200000: d_loss: 0.2483,  a_loss: 3.5563.  (1.0 sec)
22250/200000: d_loss: 0.2105,  a_loss: 5.0966.  (1.0 sec)
22300/200000: d_loss: 0.1562,  a_loss: 4.0998.  (1.0 sec)
22350/200000: d_loss: 0.3080,  a_loss: 3.2687.  (1.0 sec)
22400/200000: 

28650/200000: d_loss: 0.3974,  a_loss: 2.6199.  (1.0 sec)
28700/200000: d_loss: 0.2608,  a_loss: 3.6723.  (0.6 sec)
28750/200000: d_loss: 0.7103,  a_loss: 7.3586.  (1.0 sec)
28800/200000: d_loss: 0.3017,  a_loss: 3.5005.  (1.2 sec)
28850/200000: d_loss: 0.2833,  a_loss: 4.3291.  (1.0 sec)
28900/200000: d_loss: 0.2951,  a_loss: 3.5218.  (1.0 sec)
28950/200000: d_loss: 0.2439,  a_loss: 5.2983.  (1.0 sec)
29000/200000: d_loss: 0.3367,  a_loss: 2.8215.  (1.0 sec)
29050/200000: d_loss: 0.1562,  a_loss: 6.9982.  (1.0 sec)
29100/200000: d_loss: 0.1193,  a_loss: 4.8846.  (1.0 sec)
29150/200000: d_loss: 0.3502,  a_loss: 5.9412.  (1.0 sec)
29200/200000: d_loss: 0.2621,  a_loss: 4.2362.  (1.0 sec)
29250/200000: d_loss: 0.6444,  a_loss: 2.5413.  (1.0 sec)
29300/200000: d_loss: 0.4892,  a_loss: 3.4942.  (1.0 sec)
29350/200000: d_loss: 0.2468,  a_loss: 5.3575.  (1.0 sec)
29400/200000: d_loss: 0.3413,  a_loss: 6.2272.  (1.0 sec)
29450/200000: d_loss: 0.1961,  a_loss: 4.0754.  (1.0 sec)
29500/200000: 

...

134400/200000: d_loss: 0.9241,  a_loss: 3.2858.  (1.0 sec)
134450/200000: d_loss: 1.1781,  a_loss: 9.7344.  (1.0 sec)
134500/200000: d_loss: 0.7932,  a_loss: 4.7386.  (1.0 sec)
134550/200000: d_loss: 0.4363,  a_loss: 2.8381.  (0.7 sec)
134600/200000: d_loss: 3.2507,  a_loss: 10.1264.  (1.0 sec)
134650/200000: d_loss: 0.2405,  a_loss: 5.6610.  (1.0 sec)
134700/200000: d_loss: 2.2169,  a_loss: 1.3303.  (1.0 sec)
134750/200000: d_loss: 0.5879,  a_loss: 6.9605.  (1.0 sec)
134800/200000: d_loss: 1.1613,  a_loss: 2.9091.  (1.0 sec)
134850/200000: d_loss: 0.3364,  a_loss: 5.0893.  (1.0 sec)
134900/200000: d_loss: 1.3896,  a_loss: 7.4560.  (0.6 sec)
134950/200000: d_loss: 0.5229,  a_loss: 11.0633.  (1.0 sec)
135000/200000: d_loss: 0.8703,  a_loss: 10.1239.  (1.0 sec)
135050/200000: d_loss: 1.1056,  a_loss: 2.9847.  (1.0 sec)
135100/200000: d_loss: 0.6180,  a_loss: 5.2579.  (1.0 sec)
135150/200000: d_loss: 0.2549,  a_loss: 5.2063.  (1.0 sec)
135200/200000: d_loss: 0.8993,  a_loss: 7.8950.  (1.0

141350/200000: d_loss: 2.0496,  a_loss: 9.3524.  (1.0 sec)
141400/200000: d_loss: 0.2352,  a_loss: 9.4954.  (1.0 sec)
141450/200000: d_loss: 0.4832,  a_loss: 5.6656.  (1.0 sec)
141500/200000: d_loss: 1.5532,  a_loss: 2.9942.  (1.0 sec)
141550/200000: d_loss: 1.4853,  a_loss: 12.3891.  (1.0 sec)
141600/200000: d_loss: 0.4033,  a_loss: 5.0009.  (1.0 sec)
141650/200000: d_loss: 0.7027,  a_loss: 0.4826.  (1.0 sec)
141700/200000: d_loss: 0.7630,  a_loss: 8.4265.  (1.0 sec)
141750/200000: d_loss: 0.3766,  a_loss: 2.8323.  (1.0 sec)
141800/200000: d_loss: 0.4615,  a_loss: 7.8323.  (1.0 sec)
141850/200000: d_loss: 0.2833,  a_loss: 4.9188.  (1.0 sec)
141900/200000: d_loss: 1.1421,  a_loss: 3.9248.  (0.6 sec)
141950/200000: d_loss: 0.7875,  a_loss: 10.0683.  (1.0 sec)
142000/200000: d_loss: 0.6985,  a_loss: 4.3460.  (1.0 sec)
142050/200000: d_loss: 0.7999,  a_loss: 8.9564.  (1.0 sec)
142100/200000: d_loss: 1.0015,  a_loss: 3.0656.  (1.0 sec)
142150/200000: d_loss: 0.6268,  a_loss: 11.9137.  (1.0

148300/200000: d_loss: 0.5823,  a_loss: 6.0500.  (1.0 sec)
148350/200000: d_loss: 0.9518,  a_loss: 3.2947.  (1.0 sec)
148400/200000: d_loss: 1.4751,  a_loss: 14.9374.  (0.9 sec)
148450/200000: d_loss: 1.6144,  a_loss: 3.3492.  (1.0 sec)
148500/200000: d_loss: 0.5194,  a_loss: 16.8274.  (1.0 sec)
148550/200000: d_loss: 0.6948,  a_loss: 4.4197.  (1.0 sec)
148600/200000: d_loss: 0.5662,  a_loss: 4.0221.  (1.0 sec)
148650/200000: d_loss: 0.7080,  a_loss: 2.9204.  (1.0 sec)
148700/200000: d_loss: 0.1572,  a_loss: 5.0346.  (1.0 sec)
148750/200000: d_loss: 2.1226,  a_loss: 1.1442.  (1.0 sec)
148800/200000: d_loss: 1.3034,  a_loss: 3.0901.  (1.0 sec)
148850/200000: d_loss: 1.2761,  a_loss: 1.6932.  (2.1 sec)
148900/200000: d_loss: 0.3344,  a_loss: 5.0528.  (0.9 sec)
148950/200000: d_loss: 0.8418,  a_loss: 6.5795.  (1.0 sec)
149000/200000: d_loss: 1.5264,  a_loss: 0.0398.  (1.0 sec)
149050/200000: d_loss: 1.0443,  a_loss: 5.0341.  (2.0 sec)
149100/200000: d_loss: 0.3359,  a_loss: 6.4440.  (1.0 

155250/200000: d_loss: 0.3949,  a_loss: 7.7145.  (1.0 sec)
155300/200000: d_loss: 0.4954,  a_loss: 4.5658.  (1.0 sec)
155350/200000: d_loss: 0.5631,  a_loss: 3.4938.  (1.0 sec)
155400/200000: d_loss: 2.0289,  a_loss: 11.5621.  (1.0 sec)
155450/200000: d_loss: 0.2854,  a_loss: 4.8140.  (1.0 sec)
155500/200000: d_loss: 1.2071,  a_loss: 5.3062.  (1.0 sec)
155550/200000: d_loss: 2.4597,  a_loss: 0.6467.  (1.0 sec)
155600/200000: d_loss: 0.4419,  a_loss: 8.5749.  (0.6 sec)
155650/200000: d_loss: 1.4942,  a_loss: 11.1273.  (1.0 sec)
155700/200000: d_loss: 2.0060,  a_loss: 14.0271.  (1.0 sec)
155750/200000: d_loss: 1.4048,  a_loss: 1.0642.  (1.0 sec)
155800/200000: d_loss: 1.1966,  a_loss: 1.1877.  (1.0 sec)
155850/200000: d_loss: 3.2810,  a_loss: 2.9384.  (1.0 sec)
155900/200000: d_loss: 1.2913,  a_loss: 3.0462.  (1.0 sec)
155950/200000: d_loss: 2.6541,  a_loss: 12.6541.  (0.6 sec)
156000/200000: d_loss: 0.4732,  a_loss: 3.8510.  (1.0 sec)
156050/200000: d_loss: 0.5615,  a_loss: 1.4856.  (1.

162200/200000: d_loss: 0.5798,  a_loss: 2.0614.  (1.0 sec)
162250/200000: d_loss: 0.4484,  a_loss: 3.6143.  (1.0 sec)
162300/200000: d_loss: 0.4626,  a_loss: 2.8485.  (0.6 sec)
162350/200000: d_loss: 0.3714,  a_loss: 7.0363.  (1.0 sec)
162400/200000: d_loss: 0.5783,  a_loss: 6.4540.  (1.0 sec)
162450/200000: d_loss: 2.1306,  a_loss: 2.8718.  (1.0 sec)
162500/200000: d_loss: 1.6565,  a_loss: 4.2660.  (1.0 sec)
162550/200000: d_loss: 1.1847,  a_loss: 1.8813.  (1.0 sec)
162600/200000: d_loss: 0.5779,  a_loss: 19.8148.  (1.0 sec)
162650/200000: d_loss: 0.1810,  a_loss: 8.5637.  (1.0 sec)
162700/200000: d_loss: 0.4199,  a_loss: 8.0266.  (1.0 sec)
162750/200000: d_loss: 0.1909,  a_loss: 6.0367.  (1.0 sec)
162800/200000: d_loss: 1.0111,  a_loss: 0.7829.  (1.0 sec)
162850/200000: d_loss: 0.3826,  a_loss: 6.3256.  (1.0 sec)
162900/200000: d_loss: 0.9180,  a_loss: 6.0779.  (1.0 sec)
162950/200000: d_loss: 0.2673,  a_loss: 7.6543.  (1.0 sec)
163000/200000: d_loss: 0.6220,  a_loss: 5.6347.  (0.9 s

In [ ]:
plt.figure(1, figsize=(12, 8))
plt.subplot(121)
plt.plot(d_losses)
plt.xlabel('epochs')
plt.ylabel('discriminant losses')
plt.subplot(122)
plt.plot(a_losses)
plt.xlabel('epochs')
plt.ylabel('adverssary losses')
plt.show()

In [ ]:
import imageio
import shutil

images_to_gif = []
for filename in os.listdir(RES_DIR):
    images_to_gif.append(imageio.imread(RES_DIR + '/' + filename))
imageio.mimsave('trainnig_visual.gif', images_to_gif)
#shutil.rmtree(RES_DIR)